In [332]:
import pandas as pd

In [333]:
folder = '.\\input_files\\'

# file = '46-0319070_RapidCityHospital_StandardCharges.xlsx'
# file = '46-0360899_CusterHospital_StandardCharges.xlsx'
# file = '46-0360899_LeadDeadwoodHospital_StandardCharges.xlsx'
file = '46-0360899_SpearfishHospital_StandardCharges.xlsx'
# file = '46-0360899_SturgisHospital_StandardCharges.xlsx'

In [334]:
df = pd.read_excel(folder + file, dtype=str, skiprows=1, sheet_name='Chargemaster')

In [335]:
df.columns

Index(['Location', 'Code Type ', 'Code ', 'Procedure Code', 'Description',
       'NCD Code', 'Charge Amount', 'Cash Price', 'Uninsured Discount',
       'Aetna', 'Avera Health Plan',
       'BCBS Wellmark\nBCBS Federal\nOutpatient\n Speciality Meds',
       'Midlands Choice', 'Cigna\nCigna-Midlands Choice\nCigna-Allegiance\n',
       'National Association of Letter Carriers',
       'Allied Benefits Systems Inc', 'Key Benefits Adminstrators',
       'Meritian Health', 'Perferred One', 'Corsource\nNGS',
       'DakotaCare\nInpatient', 'DakotaCare\nOutpatient',
       'DakotaCare \nEmergency Department Services', 'FedMed',
       'First Choice Health Network', 'Employee Benefit Management',
       'Meritian Health.1', 'Pacificsource Health Plans', 'HealthPartners',
       'HealthPartners-Monument Outpatient',
       'HealthPartners - Monument\nLab is the only service ',
       'Humana Commercial', 'Multiplan Inc', 'Preferred One',
       'Group & Pension Administrators Inc', 'Medi-Share

In [336]:
df

,Location,Code Type,Code,Procedure Code,Description,NCD Code,Charge Amount,Cash Price,Uninsured Discount,Aetna,...,Medica\nCommercial,Sanford,United Heatlhcare (UHC) \nNew Groups,United Heatlhcare (UHC) \nOld Groups,United Medical Resources (UMR)\nNew Groups,United Medical Resources (UMR)\nOld Groups,Govt Employees Health Assn\nNew Groups,Govt Employees Health Assn\nOld Groups,De-Identified Minimum Negotiated Charge,De-Identified Maximum Negotiated Charge
0,Monument Health Spearfish Hospital,Room Charge,NaN,1110000100,HC ROOM PRIVATE W TELE,NaN,3080,3080,1848,2864.4,...,2926,2926,2032.8,2618,2032.8,2618,2032.8,2618,1540,2987.6
1,Monument Health Spearfish Hospital,Room Charge,NaN,1110000200,HC ROOM PRIVATE,NaN,2280,2280,1368,2120.4,...,2166,2166,1504.8,1938,1504.8,1938,1504.8,1938,1140,2211.6
2,Monument Health Spearfish Hospital,Room Charge,NaN,1120000100,HC ROOM PRIVATE OBSTETRICS,NaN,2280,2280,1368,2120.4,...,2166,2166,1504.8,1938,1504.8,1938,1504.8,1938,1140,2211.6
3,Monument Health Spearfish Hospital,Room Charge,NaN,1130000200,HC ROOM PRIVATE PEDIATRICS,NaN,2280,2280,1368,2120.4,...,2166,2166,1504.8,1938,1504.8,1938,1504.8,1938,1140,2211.6
4,Monument Health Spearfish Hospital,Room Charge,NaN,1210000100,HC ROOM SEMI PRIVATE W TELE,NaN,3080,3080,1848,2864.4,...,2926,2926,2032.8,2618,2032.8,2618,2032.8,2618,1540,2987.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,Monument Health Spearfish Hospital,CPT,0144A,7710144A00,HC IMM ADM SARSCOV2 VAC 25 MCG/0.25 ML IM (BIV...,NaN,80,80,48,74.4,...,76,76,52.8,68,52.8,68,52.8,68,48.8,77.6
3042,Monument Health Spearfish Hospital,CPT,0154A,7710154A00,HC IMM ADM SARSCOV2 VAC 10 MCG/0.2 ML IM (BIVA...,NaN,80,80,48,74.4,...,76,76,52.8,68,52.8,68,52.8,68,48.8,77.6
3043,Monument Health Spearfish Hospital,CPT,0164A,7710164A00,HC IMM ADM SRSCV2 BVL 10MCG/0.2ML (BIVALENT BO...,NaN,80,80,48,74.4,...,76,76,52.8,68,52.8,68,52.8,68,48.8,77.6
3044,Monument Health Spearfish Hospital,CPT,0173A,7710173A00,HC IMM ADM SARSCV2 BVL 3MCG/0.2ML 3,NaN,80,80,48,74.4,...,76,76,52.8,68,52.8,68,52.8,68,48.8,77.6


In [337]:
df.drop(columns='Location', inplace=True)

In [338]:
df.rename(columns={
    'Code Type ': 'line_type',
    'Code Type': 'line_type',
    'Code Type  ': 'line_type',
    'Code': 'code',
    'Code ': 'code',
    'Code   ': 'code',
    'Procedure Code': 'local_code',
    'Description': 'description',
    'NCD Code': 'ndc'

}, inplace=True)

In [339]:
cols = df.columns.tolist()
id_vars = cols[:5]
val_vars = cols[5:]

df = pd.melt(df, id_vars=id_vars, value_vars=val_vars, var_name='payer_name', value_name='standard_charge')

In [340]:
df.dropna(subset='standard_charge', inplace=True)

In [341]:
df.loc[~df['code'].isna() & df['code'].str.match(r'^[A-Z][0-9]{4}$|^[0-9]{5}$|^[0-9]{4}[A-Z]$'), 'hcpcs_cpt'] = df['code']

In [342]:
df.loc[df['payer_name'].str.contains('Outpatient|outpatient'), 'setting'] = 'outpatient'
df.loc[df['payer_name'].str.contains('Inpatient|outpatient'), 'setting'] = 'inpatient'

In [343]:
payer_mapping = {
    'Charge Amount': 'gross',
    'Cash Price': 'cash',
    'Uninsured Discount': 'cash',
    'De-Identified Minimum Negotiated Charge': 'min',
    'De-Identified Maximum Negotiated Charge': 'max'
}

df['rate_category'] = df['payer_name'].map(payer_mapping).fillna('negotiated')

In [344]:
id_mapping = {
 '46-0360899_SpearfishHospital_StandardCharges.xlsx': '430048',
 '46-0319070_RapidCityHospital_StandardCharges.xlsx': '430077',
 '46-0360899_LeadDeadwoodHospital_StandardCharges.xlsx': '431320',
 '46-0360899_SturgisHospital_StandardCharges.xlsx': '431321',
 '46-0360899_CusterHospital_StandardCharges.xlsx': '431323'
}

hosp_id = id_mapping[file]

df['hospital_id'] = hosp_id

output_folder = '.\\output_files\\'

df.to_csv(output_folder + hosp_id + '_' + file.split('_')[1] + '.csv', index=False)